<a href="https://colab.research.google.com/github/gitsDhokla/NeuralStyleTransfer/blob/main/NST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision.utils import save_image
import torchvision.transforms as transforms
import torchvision.models as models

In [ ]:
model = models.vgg19(pretrained=True).features

In [ ]:
model

In [ ]:
#imp cnn layers = [0 , 5 , 10, 19, 28]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available else 'gpu')
img_size = 512

In [ ]:
loader = transforms.Compose(
    [transforms.Resize((img_size, img_size)),
     transforms.ToTensor(),
    ]
)

In [ ]:
class VGG(nn.Module):
  def __init__(self):
    super(VGG, self).__init__()
    self.chosen_features = ['0' , '5' , '10', '19', '28']
    self.model = models.vgg19(pretrained=True).features[:29]

  def forward(self, x):
    features = []
    for layer_num, layer in enumerate(self.model):
      x = layer(x)
      if str(layer_num) in self.chosen_features:
        features.append(x)
    return features

In [ ]:
model = VGG().to(device).eval()
def load_img(name:str):
    img = Image.open(name)
    img = loader(img).unsqueeze(0)
    return img.to(device)
  
og = load_img("/content/style2.jpg")
style = load_img("/content/style6.jpg")
generated = og.clone().requires_grad_(True)
#Hyperparameters
total_steps = 6000
lr = 0.01
alpha = 1.3
beta = 0.01
optimizer = optim.Adam([generated], lr = lr)

for step in range(total_steps):
  generated_features = model(generated)
  og_features = model(og)
  style_features = model(style)
  style_loss = og_loss = 0
  for gen_feat, og_feat, style_feat in zip(
    generated_features,
    og_features,
    style_features
  ):
    batch_size, channel, height, width = gen_feat.shape
    og_loss += torch.mean((gen_feat-og_feat)**2)
      
    #Gram Matrix
    G = gen_feat.view(channel, height*width).mm(
      gen_feat.view(channel, height*width).t()
    )
    A = style_feat.view(channel, height*width).mm(
      style_feat.view(channel, height*width).t()
    )
    style_loss += torch.mean((G-A)**2)
  total_loss = alpha*og_loss + beta*style_loss
  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()

  if step % 100 == 0:
    print(total_loss)
    save_image(generated, "generated.jpg")

tensor(571628.9375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3872.8291, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1370.6223, device='cuda:0', grad_fn=<AddBackward0>)
tensor(983.8555, device='cuda:0', grad_fn=<AddBackward0>)
tensor(829.0154, device='cuda:0', grad_fn=<AddBackward0>)
tensor(723.9180, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2786.0916, device='cuda:0', grad_fn=<AddBackward0>)
tensor(617.4029, device='cuda:0', grad_fn=<AddBackward0>)
tensor(619.7510, device='cuda:0', grad_fn=<AddBackward0>)
tensor(576.1704, device='cuda:0', grad_fn=<AddBackward0>)
tensor(523.7706, device='cuda:0', grad_fn=<AddBackward0>)
tensor(475.6557, device='cuda:0', grad_fn=<AddBackward0>)
tensor(639.6917, device='cuda:0', grad_fn=<AddBackward0>)
tensor(427.4755, device='cuda:0', grad_fn=<AddBackward0>)
tensor(888.0744, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: ignored